In [ ]:
# nnUNetv2_predict \
#   -i /path/to/nnUNet_raw/DatasetXXX/imagesTs \
#   -o /path/to/nnunet_output \
#   -d DatasetXXX \
#   -c 3d_fullres \
#   -f all \
#   --save_probabilities



In [ ]:
!python3 nnunet_to_medsam2_prompts.py \
    --nnunet_output nnunet_output \
    --ct_dir ct_images \
    --out_dir prompts_out \
    --dilation_iters 3 \
    --bbox_padding 5


In [ ]:
# Minimal MedSAM2 inference: refine key-slice masks only
# Uses prompts from nnunet_to_medsam2_prompts.py and writes refined masks to medsam2_results
!python3 medsam2_infer_3D_CT_minimal.py \
    --checkpoint MedSAM2/checkpoints/MedSAM2_CTLesion.pt \
    --cfg configs/sam2.1_hiera_t512.yaml \
    --prompts_dir prompts_out \
    --output_dir medsam2_results

In [ ]:
# Stitch individual MedSAM2 segmentations into combined multi-label files
!python3 stitch_medsam2_segmentations.py \
    --masks_dir medsam2_results \
    --output_dir medsam2_results \
    --reference_dir ct_images


## Alternative: Negative Prompts Workflow

The following cells use positive and negative mask prompts for more constrained refinement.


In [ ]:
# Generate positive and negative mask prompts from nnUNet segmentations
# This creates separate positive (target label) and negative (other labels) masks
!python3 nnunet_to_medsam2_prompts_masks.py \
    --nnunet_output nnunet_output \
    --ct_dir ct_images \
    --out_dir prompts_out_masks \
    --dilation_iters 3


In [15]:
# MedSAM2 inference with positive and negative prompts
# Uses positive mask to segment target, negative point prompts to exclude other labels
!python3 medsam2_infer_3D_CT_negprompts.py \
    --checkpoint MedSAM2/checkpoints/MedSAM2_CTLesion.pt \
    --cfg configs/sam2.1_hiera_t512.yaml \
    --prompts_dir prompts_out_masks \
    --output_dir medsam2_results_negprompts


Using device: cpu for MedSAM2 inference with negative prompts
Found 66 case directories with prompts.json
Cases:   0%|                                             | 0/66 [00:00<?, ?it/s]
Processing case ct_1083
  Label LV: pos=prompts_out_masks/ct_1083.nii/LV_positive_mask.nii.gz, neg=prompts_out_masks/ct_1083.nii/LV_negative_mask.nii.gz
    Refining 93 slices with negative prompts (z=37..129)
/Users/saschastocker/Documents/Stanford/Akshay/nnUNetandMedSAM/MedSAM2/sam2/sam2_video_predictor_npz.py:965: UserWarning: dlopen(/Users/saschastocker/Documents/Stanford/Akshay/nnUNetandMedSAM/MedSAM2/sam2/_C.so, 0x0002): tried: '/Users/saschastocker/Documents/Stanford/Akshay/nnUNetandMedSAM/MedSAM2/sam2/_C.so' (slice is not valid mach-o file), '/System/Volumes/Preboot/Cryptexes/OS/Users/saschastocker/Documents/Stanford/Akshay/nnUNetandMedSAM/MedSAM2/sam2/_C.so' (no such file), '/Users/saschastocker/Documents/Stanford/Akshay/nnUNetandMedSAM/MedSAM2/sam2/_C.so' (slice is not valid mach-o file)

Ski

In [16]:
# Stitch individual negative prompts masks into combined multi-label files
# Note: --use_negprompts flag tells the script to look for *_negprompts_mask.nii.gz files
!python3 stitch_medsam2_segmentations.py \
    --masks_dir medsam2_results_negprompts \
    --output_dir medsam2_results_negprompts \
    --reference_dir ct_images \
    --use_negprompts


Found 0 cases to process: []
Processing cases: 0it [00:00, ?it/s]

Processing complete!
  Successfully processed: 0/0 cases
  Output directory: medsam2_results_negprompts
